In [1]:
import os

bundle_path = os.path.abspath("bundle-with-intermediate.pem")
os.environ["NODE_EXTRA_CA_CERTS"] = bundle_path

print("NODE_EXTRA_CA_CERTS =", os.environ["NODE_EXTRA_CA_CERTS"])

NODE_EXTRA_CA_CERTS = /home/lucas/Projects/car_dealership/bundle-with-intermediate.pem


In [ ]:
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from playwright.async_api import async_playwright

BASE = "https://www.freitasleiloeiro.com.br"


def parse_items(html: str):
    soup = BeautifulSoup(html, "lxml")
    items = []

    for card in soup.select("div.cardlote"):
        lote_el = card.select_one(".cardLote-lote")
        title_el = card.select_one(".cardLote-descVeic span")
        value_el = card.select_one(".cardLote-vlr")
        link_el = card.select_one('a[href*="LoteDetalhes"]')

        items.append({
            "lote": lote_el.get_text(strip=True) if lote_el else None,
            "title": title_el.get_text(" ", strip=True) if title_el else None,
            "value": value_el.get_text(" ", strip=True) if value_el else None,
            "details_url": urljoin(BASE, link_el["href"]) if link_el else None,
        })

    return items


async def fetch_all_lots(leilao_id=7659, top_rows=12, max_pages=50):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(
            locale="pt-BR",
            user_agent=(
                "Mozilla/5.0 (X11; Ubuntu; Linux x86_64) "
                "AppleWebKit/537.36 (KHTML, like Gecko) "
                "Chrome/121.0 Safari/537.36"
            ),
        )
        page = await context.new_page()

        # Step 1: entry page (passes gocache/WAF)
        await page.goto(
            f"{BASE}/Leiloes/Lotes?leilaoId={leilao_id}",
            wait_until="domcontentloaded",
        )
        await page.wait_for_timeout(1500)

        all_items = []

        # Step 2: paginate ListarLotes
        for page_number in range(1, max_pages + 1):
            url = (
                f"{BASE}/Leiloes/ListarLotes"
                f"?LeilaoId={leilao_id}"
                f"&Nome=&Lote=&Categoria=1&TipoLoteId=0&FaixaValor=0"
                f"&Condicao=&PatioId=0&AnoModeloMin=0&AnoModeloMax=0"
                f"&ArCondicionado=false&DirecaoAssistida=false"
                f"&Tag=&ClienteSclId=0"
                f"&PageNumber={page_number}&TopRows={top_rows}"
            )

            resp = await page.request.get(
                url,
                headers={
                    "X-Requested-With": "XMLHttpRequest",
                    "Referer": f"{BASE}/Leiloes/Lotes?leilaoId={leilao_id}",
                    "Accept": "text/html,*/*;q=0.8",
                },
            )

            if resp.status != 200:
                print(f"Stopped at page {page_number} (status {resp.status})")
                break

            html = await resp.text()
            items = parse_items(html)

            if not items:
                break

            all_items.extend(items)

            # polite pacing
            await page.wait_for_timeout(600)

        await browser.close()
        return all_items

In [3]:
items = await fetch_all_lots(leilao_id=7659, max_pages=1)
len(items)

12

In [4]:
items

[{'lote': '001',
  'title': 'I/FORD FOCUS SE AT 1.6HB, 15/15, PLACA: P__-___6, GASOL/ALC, PRATA',
  'value': 'R$ 26.000,00',
  'details_url': 'https://www.freitasleiloeiro.com.br/Leiloes/LoteDetalhes?leilaoId=7659&loteNumero=1'},
 {'lote': '002',
  'title': 'CHEV/ONIX 10MT LT2, 20/21, PLACA: R__-___3, GASOL/ALC, BRANCA',
  'value': 'R$ 32.000,00',
  'details_url': 'https://www.freitasleiloeiro.com.br/Leiloes/LoteDetalhes?leilaoId=7659&loteNumero=2'},
 {'lote': '003',
  'title': 'VW/GOLF 1.6 SPORTLINE, 11/12, PLACA: E__-___0, GASOL/ALC, PRETA',
  'value': 'R$ 32.000,00',
  'details_url': 'https://www.freitasleiloeiro.com.br/Leiloes/LoteDetalhes?leilaoId=7659&loteNumero=3'},
 {'lote': '004',
  'title': 'RENAULT/KWID ZEN 2, 22/23, PLACA: R__-___8, GASOL/ALC, BRANCA',
  'value': 'R$ 26.000,00',
  'details_url': 'https://www.freitasleiloeiro.com.br/Leiloes/LoteDetalhes?leilaoId=7659&loteNumero=4'},
 {'lote': '005',
  'title': 'FIAT/ARGO TREKKING 1.3AT, 23/24, PLACA: S__-___9, GASOL/ALC, PRE